# SETTINGS

## Libraries

In [1]:

import fitz
import os
import shutil
import re
import pandas as pd
import numpy as np
os.getcwd()

'C:\\Users\\juani\\Documents\\3_My_Jupiter_Notebooks\\5_Galicia\\7_boletines'

## Loading Files

In [61]:
#levantamos el documento
my_path = r'data/seccion_segunda_20230719.pdf'
doc = fitz.open(my_path)

In [62]:
# juntamos las páginas:
text = ''
for n,page in enumerate(doc):
    if n >1:
        text += page.get_text()

## Preprocessing

In [ ]:
def prepro(text):
    ''' 
    Función de preprocesamiento.
    '''
    pattern = r'BOLETÍN OFICIAL Nº.*?de \d{4}'
    matches = re.findall(pattern, text, re.DOTALL)
    # eliminamos las cabeceras con fecha y numero de boletín:
    for match in matches:
        text = text.replace(match, '')
    # Eliminamos los #F6917090F#
    pattern = r'#[IF]\d+I#|#[IF]\d+F#\n'
    text = re.sub(pattern, '', text)
    return text

In [89]:
# eliminamos el zócalo de
pattern = r'BOLETÍN OFICIAL Nº.*?de \d{4}'
matches = re.findall(pattern, text, re.DOTALL)
len(matches) , matches[0]

(82,
 'BOLETÍN OFICIAL Nº 35.214 - Segunda Sección \n3 \nMiércoles 19 de julio de 2023')

In [90]:
# Remove the matches from the original text
for match in matches:
    text = text.replace(match, '')
    
# Eliminamos los #F6917090F#
pattern = r'#[IF]\d+I#|#[IF]\d+F#\n'
text = re.sub(pattern, '', text)

----

In [91]:
# separamos los títulos

def split_by_chapter(text):
    headers = ['CONTRATOS SOBRE PERSONAS JURÍDICAS', 'CONVOCATORIAS Y AVISOS COMERCIALES', 'EDICTOS JUDICIALES']
    pattern = '|'.join(re.escape(header) for header in headers)
    chapters = re.split(pattern, text)

    # Remove empty strings from the result
    chapters = [chapter.strip() for chapter in chapters if chapter.strip()]

    return chapters


chapters = split_by_chapter(text)
len(chapters)

5

In [99]:
# obtenemos los nombres de los separadores:
headers = ['CONTRATOS SOBRE PERSONAS JURÍDICAS', 'CONVOCATORIAS Y AVISOS COMERCIALES', 'EDICTOS JUDICIALES']
pattern = '|'.join(re.escape(header) for header in headers)
chapter_name = re.findall(pattern, text)
chapter_name

['CONTRATOS SOBRE PERSONAS JURÍDICAS',
 'CONVOCATORIAS Y AVISOS COMERCIALES',
 'EDICTOS JUDICIALES',
 'CONVOCATORIAS Y AVISOS COMERCIALES',
 'EDICTOS JUDICIALES']

# Creamos el DataFrame

In [129]:
separator_regex = r'e\.\s\d{2}/\d{2}/\d{4}\sN°\s\d+/\d+\sv\.\s\d{2}/\d{2}/\d{4}'
sep_col = []
ch_col = []
part_col = []

for n,chapter in enumerate(chapters):    
    separators = re.findall(separator_regex, chapter)
    parts = re.split(separator_regex, chapter)
    parts = [l for l in parts if bool(l)]
    if len(parts)==len(separators):
        sep_col.extend(separators)
        part_col.extend(parts)
        ch_n = [chapter_name[n] for m in range(len(parts))]
        ch_col.extend(ch_n)
        

In [132]:
df = pd.DataFrame({'chapter':ch_col,'escrito':sep_col,'texto':part_col,})
df.head()

,chapter,escrito,texto
0,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55783/23 v. 19/07/2023,SOCIEDADES ANÓNIMAS\nACEGO S.A.\nEscritura Nº ...
1,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55773/23 v. 19/07/2023,\nAGROQUIM NORTE S.A.\nEsc 52 del 03/07/2023 F...
2,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55779/23 v. 19/07/2023,\n\n\n \nALCARI S.A.\nNúmero Correlativo 1.707...
3,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55528/23 v. 19/07/2023,\nB2FI S.A.\n1) 17/07/2023. 2) Francisco LEVIS...
4,CONTRATOS SOBRE PERSONAS JURÍDICAS,e. 19/07/2023 N° 55539/23 v. 19/07/2023,\nCMIX ARGENTINA S.A.\nCUIT: 30-71088679-9.Com...


In [175]:
os.getcwd()

'C:\\Users\\juani\\Documents\\3_My_Jupiter_Notebooks\\5_Galicia\\7_boletines'

# FUNCION QUE RESUME EL TRABAJO ANTERIOR

In [87]:
#levantamos el documento
my_path = r'data/seccion_segunda_20230719.pdf'
doc = fitz.open(my_path)

In [2]:
# separamos los títulos
def split_by_chapter(text):
    headers = ['CONTRATOS SOBRE PERSONAS JURÍDICAS', 'CONVOCATORIAS Y AVISOS COMERCIALES', 'EDICTOS JUDICIALES']
    pattern = '|'.join(re.escape(header) for header in headers)
    chapters = re.split(pattern, text)

    # Remove empty strings from the result
    chapters = [chapter.strip() for chapter in chapters if chapter.strip()]

    return chapters


def parsear(doc):
    '''
    Parsea un pdf levantado con fitz
    '''
    
    # PREPROCESAMIENTO:
    
    text = ''
    for n,page in enumerate(doc):
        if n >1:
            text += page.get_text()
            
    # Eliminamos los #F6917090F#
    pattern = r'#[IF]\d+I#|#[IF]\d+F#\n'
    text = re.sub(pattern, '', text)
    
    pattern = r'BOLETÍN OFICIAL Nº.*?de \d{4}'
    matches = re.findall(pattern, text, re.DOTALL)
    # Remove the matches from the original text
    for match in matches:
        text = text.replace(match, '')
     
    # partimos en chapters:
    chapters = split_by_chapter(text)
    #print(f"# chapters: {len(chapters)}")
    
    # obtenemos los nombres de los separadores:
    headers = ['CONTRATOS SOBRE PERSONAS JURÍDICAS', 'CONVOCATORIAS Y AVISOS COMERCIALES', 'EDICTOS JUDICIALES']
    pattern = '|'.join(re.escape(header) for header in headers)
    chapter_name = re.findall(pattern, text)
    
    
    separator_regex = r'e\.\s\d{2}/\d{2}/\d{4}\sN°\s\d+/\d+\sv\.\s\d{2}/\d{2}/\d{4}'
    sep_col = []
    ch_col = []
    part_col = []

    for n,chapter in enumerate(chapters):    
        separators = re.findall(separator_regex, chapter)
        parts = re.split(separator_regex, chapter)
        parts = [l for l in parts if bool(l)]
        if len(parts)==len(separators):
            sep_col.extend(separators)
            part_col.extend(parts)
            ch_n = [chapter_name[n] for m in range(len(parts))]
            ch_col.extend(ch_n)
            
    temp = pd.DataFrame({'chapter':ch_col,'escrito':sep_col,'texto':part_col})
    return temp

In [3]:
for file in sel_bol_23[0:1]:
    doc = fitz.open(f'data\\{file}')
    temp = parsear(doc)

NameError: name 'sel_bol_23' is not defined

In [28]:
def split_by_chapter(text):
    headers = ['CONTRATOS SOBRE PERSONAS JURÍDICAS', 'CONVOCATORIAS Y AVISOS COMERCIALES', 'EDICTOS JUDICIALES']
    pattern = '|'.join(re.escape(header) for header in headers)
    matches = re.finditer(pattern, text)

    # Extract separators from matches
    separators = [match.group(0) for match in matches]

    # Split the text using the pattern
    chapters = re.split(pattern, text)

    # Remove empty strings from the result
    chapters = [chapter.strip() for chapter in chapters if chapter.strip()]

    return chapters, separators

# Example usage:
# text = "Your input text goes here. CONTRATOS SOBRE PERSONAS JURÍDICAS Some content CONVOCATORIAS Y AVISOS COMERCIALES More content EDICTOS JUDICIALES Last part"
# chapters, separators = split_by_chapter(text)

# # Print the results
# print("Chapters:", chapters)
# print("Separators:", separators)
# In this example, I've used re.finditer() to find all matches of the pattern in the text, and then extracted the matched separators from the matches. The separators list will contain the separators corresponding to the splits in the chapters list.

# Feel free to adjust the example based on your specific needs or let me know if you have any further questions!


def parsear(doc):
    '''
    Parsea un pdf levantado con fitz
    '''
    
    # PREPROCESAMIENTO:
    
    text = ''
    for n,page in enumerate(doc):
        if n >1:
            text += page.get_text()
            
    # Eliminamos los #F6917090F#
    pattern = r'#[IF]\d+I#|#[IF]\d+F#\n'
    text = re.sub(pattern, '', text)
    
    pattern = r'BOLETÍN OFICIAL Nº.*?de \d{4}'
    matches = re.findall(pattern, text, re.DOTALL)
    # Remove the matches from the original text
    for match in matches:
        text = text.replace(match, '')
     
    # partimos en chapters:
    chapters,chapter_name = split_by_chapter(text)
    #print(f"# chapters: {len(chapters)}")
    
    # # obtenemos los nombres de los separadores:
    # headers = ['CONTRATOS SOBRE PERSONAS JURÍDICAS', 'CONVOCATORIAS Y AVISOS COMERCIALES', 'EDICTOS JUDICIALES']
    # pattern = '|'.join(re.escape(header) for header in headers)
    # chapter_name = re.findall(pattern, text)
    
    
    separator_regex = r'e\.\s\d{2}/\d{2}/\d{4}\sN°\s\d+/\d+\sv\.\s\d{2}/\d{2}/\d{4}'
    sep_col = []
    ch_col = []
    part_col = []

    for n,chapter in enumerate(chapters):    
        separators = re.findall(separator_regex, chapter)
        parts = re.split(separator_regex, chapter)
        parts = [l for l in parts if bool(l)]
        if len(parts)==len(separators):
            sep_col.extend(separators)
            part_col.extend(parts)
            ch_n = [chapter_name[n] for m in range(len(parts))]
            ch_col.extend(ch_n)
            
    temp = pd.DataFrame({'chapter':ch_col,'escrito':sep_col,'texto':part_col})
    return temp






In [38]:
def split_by_chapter(text):
    headers = ['CONTRATOS SOBRE PERSONAS JURÍDICAS', 'CONVOCATORIAS Y AVISOS COMERCIALES', 'EDICTOS JUDICIALES']
    pattern = '|'.join(re.escape(header) for header in headers)
    matches = re.finditer(pattern, text)

    # Extract separators from matches
    separators = [match.group(0) for match in matches]

    # Split the text using the pattern
    chapters = re.split(pattern, text)

    # Remove empty strings from the result
    chapters = [chapter.strip() for chapter in chapters if chapter.strip()]

    # Ensure the length of chapters is the same as the length of separators
    # min_length = min(len(chapters), len(separators))
    # chapters = chapters[:min_length]
    # separators = separators[:min_length]

    # Label 'undefined' for the remaining separators
    separators.extend(['undefined'] * (len(chapters) - len(separators)))

    return chapters, separators


In [63]:
test,headers = split_by_chapter(text)
headers

['CONTRATOS SOBRE PERSONAS JURÍDICAS',
 'CONVOCATORIAS Y AVISOS COMERCIALES',
 'EDICTOS JUDICIALES',
 'CONVOCATORIAS Y AVISOS COMERCIALES',
 'EDICTOS JUDICIALES',
 'undefined']

In [66]:
test[1]

'BOLETÍN OFICIAL Nº 35.214 - Segunda Sección \n3 \nMiércoles 19 de julio de 2023'

In [60]:
len(chapters)

4

In [39]:
doc = fitz.open('data\\'+file)
test = parsear(doc)

In [251]:
chapter_name

['CONVOCATORIAS Y AVISOS COMERCIALES',
 'EDICTOS JUDICIALES',
 'CONVOCATORIAS Y AVISOS COMERCIALES']

In [258]:
# chapters[0]

In [10]:
doc = fitz.open('data\\'+file)
text = ''
for n,page in enumerate(doc):
    if n >1:
        text += page.get_text()

# Eliminamos los #F6917090F#
pattern = r'#[IF]\d+I#|#[IF]\d+F#\n'
text = re.sub(pattern, '', text)

pattern = r'BOLETÍN OFICIAL Nº.*?de \d{4}'
matches = re.findall(pattern, text, re.DOTALL)
# Remove the matches from the original text
for match in matches:
    text = text.replace(match, '')

# partimos en chapters:
chapters = split_by_chapter(text)
#print(f"# chapters: {len(chapters)}")

# obtenemos los nombres de los separadores:
headers = ['CONTRATOS SOBRE PERSONAS JURÍDICAS', 'CONVOCATORIAS Y AVISOS COMERCIALES', 'EDICTOS JUDICIALES']
pattern = '|'.join(re.escape(header) for header in headers)
chapter_name = re.findall(pattern, text)


separator_regex = r'e\.\s\d{2}/\d{2}/\d{4}\sN°\s\d+/\d+\sv\.\s\d{2}/\d{2}/\d{4}'
sep_col = []
ch_col = []
part_col = []

for n,chapter in enumerate(chapters):    
    separators = re.findall(separator_regex, chapter)
    parts = re.split(separator_regex, chapter)
    parts = [l for l in parts if bool(l)]
    if len(parts)==len(separators):
        sep_col.extend(separators)
        part_col.extend(parts)
        ch_n = [chapter_name[n] for m in range(len(parts))]
        ch_col.extend(ch_n)

temp = pd.DataFrame({'chapter':ch_col,'escrito':sep_col,'texto':part_col})

IndexError: list index out of range

In [13]:
len(chapter)

71683

In [11]:
chapter_name

['CONVOCATORIAS Y AVISOS COMERCIALES',
 'EDICTOS JUDICIALES',
 'CONVOCATORIAS Y AVISOS COMERCIALES']

In [56]:
df.chapter.unique()

array(['CONTRATOS SOBRE PERSONAS JURÍDICAS',
       'CONVOCATORIAS Y AVISOS COMERCIALES', 'EDICTOS JUDICIALES'],
      dtype=object)

In [54]:
df.texto[0][0:1][0]

'SOCIEDADES ANÓNIMAS\nALYSER\xa0S.A.\nCUIT 30-70776023-7. Por Acta de Asamblea Extraordinaria del 31/08/2018 se reformó el Artículo Décimo Primero \ndel Estatuto modificando las siguientes estipulaciones: Se resolvió ampliar el plazo de duración del mandato de \nlos Directores de un ejercicio a tres ejercicios. Autorizado según instrumento privado Acta de Asamblea de fecha \n31/08/2018\nmonica laura conde - T°: 40 F°: 451 C.P.A.C.F.\n'

In [8]:
df = None
for n,file in enumerate(sel_bol_2):
    doc = fitz.open('data\\'+file)
    temp = parsear(doc)
    if df is not None:
        df = pd.concat([df,temp])
    else:
        df = temp.copy()
        
    print(n,end='\r')

IndexError: list index out of range

In [232]:
len(df)

362725

In [234]:
df.to_pickle('df.pkl')

In [240]:
len(df)

362725

In [57]:
len(df[df.chapter==df.chapter.unique()[0]])

4197

In [229]:
# print(df[df.chapter==df.chapter.unique()[0]].texto[0:10][5])

In [203]:
len(df)

9625

----

# Creamos el DataFrame

In [129]:
separator_regex = r'e\.\s\d{2}/\d{2}/\d{4}\sN°\s\d+/\d+\sv\.\s\d{2}/\d{2}/\d{4}'
sep_col = []
ch_col = []
part_col = []

for n,chapter in enumerate(chapters):    
    separators = re.findall(separator_regex, chapter)
    parts = re.split(separator_regex, chapter)
    parts = [l for l in parts if bool(l)]
    if len(parts)==len(separators):
        sep_col.extend(separators)
        part_col.extend(parts)
        ch_n = [chapter_name[n] for m in range(len(parts))]
        ch_col.extend(ch_n)
        

# Iteramos sobre todos los documentos de 2023

In [5]:
sel_bol_23 = [element for element in sel_bol if re.search(r'seccion_segunda_2023', element)]
len(sel_bol_23)

35

--

# BACKUP (OTRAS TAREAS)

# Copiamos los pdfs al disco:

In [4]:
# 'C:\\Users\\juani\\Documents\\3_My_Jupiter_Notebooks\\5_Galicia\\7_boletines'
source_folder = r'D:\7_boletines\data\CABA'
destination_folder = r'C:\\Users\\juani\\Documents\\3_My_Jupiter_Notebooks\\5_Galicia\\7_boletines\\data'
boletines = os.listdir(source_folder)
boletines_repetidos = [element for element in boletines if '(' in element]
len(boletines_repetidos), len(boletines)
sel_bol = [bol for bol in boletines if bol not in boletines_repetidos]
sel_bol_2 = [element for element in sel_bol if re.search(r'seccion_segunda_202', element)]
len(sel_bol_2)

1319

In [169]:
# Paths for the source and destination folders

# Iterate through the file list and copy each file
for file_name in sel_bol_2:
    source_path = os.path.join(source_folder, file_name)
    destination_path = os.path.join(destination_folder, file_name)
    
    shutil.copy2(source_path, destination_path)

print("Files copied successfully.")

Files copied successfully.


---

In [98]:
# separamos las partes por escrito:
example = chapters[0]
separator_regex = r'e\.\s\d{2}/\d{2}/\d{4}\sN°\s\d+/\d+\sv\.\s\d{2}/\d{2}/\d{4}'
separators = re.findall(separator_regex, example)
parts = re.split(separator_regex, example)
parts = [element for element in parts if bool(element)]
len(parts),len(separators)

(85, 85)

---